In [1]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models
import os
import matplotlib.pyplot as plt
from torch.optim import Adam
from tqdm import tqdm_notebook

print(torch.__version__)
device = torch.device('cuda:0')

1.2.0


In [2]:
class Weedread(Dataset):
    def __init__(self, name, transform=None):
        hf = h5py.File(name, 'r')
        input_images = np.array(hf.get('data'), np.uint8)
        target_labels = np.array(hf.get('labels')).astype(np.long)
        
        self.input_images = input_images
        self.target_labels = target_labels
        self.transform = transform
        hf.close()

    def __len__(self):
        return self.input_images.shape[0]

    def __getitem__(self, idx):
        images = self.input_images[idx]
        classes = self.target_labels[idx][1]
        family =  self.target_labels[idx][0]
        if self.transform is not None:
            images = self.transform(images)
        images = images
        
        return images, classes, family

In [3]:
INPUT_CHANNEL = 3
BATCH_SIZE = 32
normalize = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.Resize((96,96)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

imagenet_classes = range(1, 22)

data_path = os.path.dirname(os.getcwd()) + "/data/weed/"
Train_data = Weedread(data_path + "train.h5", transform=normalize)
Test_data = Weedread(data_path + "val.h5", transform=normalize)

Train_dataloader = DataLoader(dataset=Train_data,
                              batch_size = BATCH_SIZE,
                              shuffle=True)
Test_dataloader = DataLoader(dataset=Test_data,
                              batch_size = BATCH_SIZE,
                              shuffle=False)

In [4]:

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

class My_Model(nn.Module):
    def __init__(self, input_channel=1, num_class=21, num_family = 5):
        super(My_Model, self).__init__()
        model = models.resnet18(pretrained=True)
        self.model_ft = torch.nn.Sequential(*(list(model.children())[:-1]))
        print(self.model_ft)
        set_parameter_requires_grad(self.model_ft, False)
        self.average_pool = nn.AdaptiveAvgPool2d(output_size=2)

        self.family_fc = nn.Linear(2048, num_family)
        self.class_fc = nn.Linear(512, num_class)
        
    
    def forward(self, x):
        # Perform the usual forward pass
        for name, module in self.model_ft._modules.items():
            x = module(x)
            if(name == "7"):
                x_family = x
                x_family = self.average_pool(x_family)
                
        x = torch.flatten(x, 1)
        x_class = self.class_fc(x)
        
        x_family = torch.flatten(x_family, 1)
        x_family = self.family_fc(x_family)
        return F.softmax(x_class, dim=1), F.softmax(x_family, dim=1)

In [5]:
likelihood_matrix = torch.randn(5, 21).fill_(10e-6).to(device)
likelihood_matrix[0, 0:11] = 1
likelihood_matrix[1, 11] = 1
likelihood_matrix[2, 12:14] = 1
likelihood_matrix[3, 14:19] = 1
likelihood_matrix[4, 19:21] = 1

class _loss(nn.Module):
    def __init__(self, alpha=0.5):
        super(_loss, self).__init__()
        self.alpha = alpha
        self.family_loss = nn.CrossEntropyLoss()
        self.class_conditional_family_loss = nn.CrossEntropyLoss()
        
    def forward(self, predicted_class, true_class, predicted_family, true_family):
        
        F_X = predicted_family[torch.arange(predicted_family.size(0)), true_family]
        F_X = F_X.view(F_X.size(0), 1)
        Y_X = predicted_class
        F_YX = likelihood_matrix[true_family]
        
        #P(Y|F, X) = (P(F|Y, X) * P(Y|X)) /(P(F|X))
        #Y_XF = (F_YX * Y_X ) / F_X
        L = self.class_conditional_family_loss(Y_X * F_YX, true_class) - torch.log(F_X).sum(dim=0)/(BATCH_SIZE)
        return L
        #return  self.alpha*self.family_loss(predicted_family, true_family) + \
        #        (1-self.alpha)*self.class_conditional_family_loss(Y_X, true_class)
                
        

In [6]:
from torchsummary import summary
train_images, _, _ = next(iter(Test_dataloader))


_model = My_Model(num_class = 21, num_family = 5)
summary(_model, input_size= train_images[0].size(), device="cpu")
_model = _model.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [7]:
from tqdm import tqdm_notebook

optimizer = torch.optim.SGD(_model.parameters(), lr=0.01)
criterion = _loss()
EPOCHS = 100

max_correct = 0
for epoch in range(1, EPOCHS + 1):
    #training
    _model.train()
    for image, classes, family in tqdm_notebook(Train_dataloader):
        image, classes, family = image.to(device), classes.to(device), family.to(device)
        image = image.float()
        p_classes, p_family = _model(image)
        loss = criterion(p_classes, classes, p_family, family)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Loss :{:.4f} Epoch[{}/{}]'.format(loss.item(), epoch, EPOCHS))
    #testing
    _model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for image, classes, family in (Test_dataloader):
            image, classes, family = image.to(device), classes.to(device), family.to(device)
            image = image.float()
            p_classes, p_family = _model(image)
            
            max_value, max_index = torch.max(p_family, dim=1)
            F_YX = likelihood_matrix[max_index]
            F_X = max_value.view(max_value.size(0), 1)
            Y_X = p_classes
            Y_FX = (F_YX * Y_X ) / F_X
        
            predicted = torch.argmax(Y_FX, dim=1)
            total += image.size(0)
            correct += (predicted == classes).sum().item()
        print('Test Accuracy of the model on the test images: {:.4f} %'.format(100 * correct / total))
    if(correct > max_correct):
        max_correct = correct
        torch.save(_model.state_dict(), 'epochs/ResNet18-class-conditional-family1.pt')


Loss :2.5084 Epoch[1/100]
Test Accuracy of the model on the test images: 72.2799 %



Loss :2.4778 Epoch[2/100]
Test Accuracy of the model on the test images: 76.1067 %



Loss :2.3160 Epoch[3/100]
Test Accuracy of the model on the test images: 82.2712 %



Loss :2.2276 Epoch[4/100]
Test Accuracy of the model on the test images: 85.0416 %



Loss :2.3904 Epoch[5/100]
Test Accuracy of the model on the test images: 85.5842 %


...


Loss :2.1232 Epoch[96/100]
Test Accuracy of the model on the test images: 96.2425 %



Loss :2.1233 Epoch[97/100]
Test Accuracy of the model on the test images: 96.2735 %



Loss :2.1232 Epoch[98/100]
Test Accuracy of the model on the test images: 96.2449 %



Loss :2.1619 Epoch[99/100]
Test Accuracy of the model on the test images: 96.2257 %



Loss :2.1352 Epoch[100/100]
Test Accuracy of the model on the test images: 96.2090 %
